<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/BGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=47dbc8c709e65a27cec4cb44e97945e731124c170df7e6fa93f73eedc36500ac
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
!wget 'https://zenodo.org/record/8196385/files/BGL.zip'
!unzip "/content/BGL.zip" -d "/content/"

--2023-09-04 15:31:20--  https://zenodo.org/record/8196385/files/BGL.zip
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57489019 (55M) [application/octet-stream]
Saving to: ‘BGL.zip’

BGL.zip             100%[===================>]  54.83M  2.31MB/s    in 48s     

2023-09-04 15:32:10 (1.15 MB/s) - ‘BGL.zip’ saved [57489019/57489019]

Archive:  /content/BGL.zip
  inflating: /content/BGL.log        
  inflating: /content/README.md      


In [3]:
import re
import string
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
import json

In [4]:
def clean(s):
    """ Preprocess log message
    Parameters
    ----------
    s: str, raw log message

    Returns
    -------
    str, preprocessed log message without number tokens and special characters
    """
    # s = re.sub(r'(\d+\.){3}\d+(:\d+)?', " ", s)
    # s = re.sub(r'(\/.*?\.[\S:]+)', ' ', s)
    s = re.sub('\]|\[|\)|\(|\=|\,|\;', ' ', s)
    s = " ".join([word.lower() if word.isupper() else word for word in s.strip().split()])
    s = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', s))
    s = " ".join([word for word in s.split() if not bool(re.search(r'\d', word))])
    trantab = str.maketrans(dict.fromkeys(list(string.punctuation)))
    content = s.translate(trantab)
    s = " ".join([word.lower().strip() for word in content.strip().split()])
    return s

In [5]:
def split_norm_abnorm(file,windows_size,step_size):

  with open(file, mode="r", encoding='utf8') as f:
      logs = f.readlines()
      logs = [x.strip() for x in logs]

  x_normal, x_abnormal = [], []
  i = 0

  while i < len(logs) - windows_size:

      seq = []
      label = 0
      for j in range(i, i + windows_size):
          if logs[j][0] != "-":
              label = 1
          content = logs[j]
          # remove label from log messages
          content = content[content.find(' ') + 1:]
          content = clean(content.lower())

          seq.append(content)
      if label == 1:
        x_abnormal.append(seq)
      else :
        x_normal.append(seq)

      i = i + step_size

  print('len(logs) : ',len(logs))
  print('len(x_normal) : ',len(x_normal))
  print('len(x_abnormal) : ',len(x_abnormal))

  return x_normal,x_abnormal

In [6]:
windows_size=20
step_size = 20
x_normal,x_abnormal = split_norm_abnorm('/content/BGL.log',windows_size,step_size)

len(logs) :  4747963
len(x_normal) :  217137
len(x_abnormal) :  20261


In [7]:
def split_norm_data(normal_data, train_ratio):
  len_train = int(len(normal_data) * train_ratio)
  x_normal_train = normal_data[:len_train]
  x_normal_test = normal_data[len_train:]
  print('len(x_normal_train) : ',len(x_normal_train))
  print('len(x_normal_test) : ',len(x_normal_test))

  return x_normal_train, x_normal_test


In [8]:
train_ratio=0.8
x_normal_train, x_normal_test = split_norm_data(x_normal, train_ratio)

len(x_normal_train) :  173709
len(x_normal_test) :  43428


In [9]:
def structured_train(file):
  log2index = {}
  index2embed = {}
  X_train_index = []
  index = 0
  for logs in file:
    seq = []
    for log in logs:
        if log not in log2index:
            log2index[log] = index
            index2embed[index] = model.encode(log)
            seq.append(index)
            index += 1
        else:
            seq.append(log2index[log])
    X_train_index.append(seq)

  embeddings = np.array(list(index2embed.values()))
  pca = PCA(n_components=0.9)
  pca.fit(embeddings)
  reduced_embeddings = pca.transform(embeddings)

  reduced_index2embed = {}
  for i,key in enumerate(index2embed.keys()):
    reduced_index2embed[key]=reduced_embeddings[i]

  print('number of unique templates(train) : ', len(log2index))

  return log2index, reduced_index2embed, X_train_index, pca

In [10]:
log2index, reduced_index2embed, X_train_index, pca  = structured_train(x_normal_train)

number of unique templates(train) :  504


In [11]:
print('number of dementions of embedding : ', reduced_index2embed[0].shape)
print('log sequence of a sample(fixed windows size=20) : ', X_train_index[5560])

number of dementions of embedding :  (60,)
log sequence of a sample(fixed windows size=20) :  [2, 2, 5, 2, 2, 2, 5, 2, 2, 2, 5, 2, 2, 2, 5, 2, 2, 2, 5, 2]


In [15]:
def structured_test(norm_file,abnorm_file,log2index,reduced_index2embed,pca):
  Xnorm_test_index = []
  index = len(log2index)
  for logs in norm_file:
    seq = []
    for log in logs:
        if log not in log2index:
            log2index[log] = index
            embeddings = model.encode(log).reshape(1, -1) # reshape to make it 2D proper for pca.transform's input
            reduced_index2embed[index] = pca.transform(embeddings).flatten() #flatten() converts shape (1, 60) to (60,)
            seq.append(index)
            index += 1
        else:
            seq.append(log2index[log])
    Xnorm_test_index.append(seq)

  print('number of unique templates(train + normal_test) : ', len(log2index))

  Xabnorm_test_index = []
  for logs in abnorm_file:
    seq = []
    for log in logs:
        if log not in log2index:
            log2index[log] = index
            embeddings = model.encode(log).reshape(1, -1) # reshape to make it 2D proper for pca.transform's input
            reduced_index2embed[index] = pca.transform(embeddings).flatten() #flatten() converts shape (1, 60) to (60,)
            seq.append(index)
            index += 1
        else:
            seq.append(log2index[log])
    Xabnorm_test_index.append(seq)

  print('number of unique templates(train + normal_test + abnormal_test) : ', len(log2index))

  return log2index, reduced_index2embed, Xnorm_test_index, Xabnorm_test_index

In [16]:
log2index, reduced_index2embed, Xnorm_test_index, Xabnorm_test_index = structured_test(x_normal_test,x_abnormal,log2index,reduced_index2embed,pca)

number of unique templates(train + normal_test) :  638
number of unique templates(train + normal_test + abnormal_test) :  736


In [ ]:
Xabnorm_test_index[400:420]

In [ ]:
log2index

In [18]:
# we upload, in case that we cant parse logs(a lot of time and ram consumption)

# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for line in X_train_index:
  print(' '.join(str(token) for token in line))
  break

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [21]:
# save hdfs_sequence file in google drive
with open('/content/drive/MyDrive/BGL/log2index', 'w') as log_file:
  for key in log2index.keys():
      log_file.write(key + '\n')


with open('/content/drive/MyDrive/BGL/X_train_index', 'w') as log_file:
  for line in X_train_index:
    log_entry = ' '.join(str(token) for token in line)
    log_file.write(log_entry + '\n')

with open('/content/drive/MyDrive/BGL/Xnorm_test_index', 'w') as log_file:
  for line in Xnorm_test_index:
    log_entry = ' '.join(str(token) for token in line)
    log_file.write(log_entry + '\n')


with open('/content/drive/MyDrive/BGL/Xabnorm_test_index', 'w') as log_file:
  for line in Xabnorm_test_index:
    log_entry = ' '.join(str(token) for token in line)
    log_file.write(log_entry + '\n')



In [22]:
# Extract the values (lists) from the dictionary and save each as a JSON array
with open('/content/drive/MyDrive/BGL/reduced_index2embed', 'w') as file:
    for value_list in reduced_index2embed.values():
        json.dump(value_list.tolist(), file)
        file.write('\n')  # Add a newline to separate arrays